# Streaming historical data from timescaleDB

In the "Accessing TimescaleDB" notebook, I demonstrated how to connect to the TimescaleDB instance and query some data statically. Now, I'd like to do some experiments on what latency looks like for streaming data. How expensive is it to query the last hour of data? How about the last two hours of data? Are there tricks we can play to make it faster?

*Aniket Pant, UIUC*

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from datetime import datetime, timedelta

In [2]:
db_url = URL.create(
    "postgresql+psycopg2",
    username=os.getenv("POSTGRES_USER", "ships"),
    password=os.getenv("POSTGRES_PASSWORD", "ships"),
    host=os.getenv("POSTGRES_HOST", "localhost"),
    port=int(os.getenv("POSTGRES_PORT", 5432)),
    database=os.getenv("POSTGRES_DB", "ships"),
)

engine = create_engine(db_url, future=True)

with engine.connect() as connection:
    df = pd.read_sql(text("SELECT NOW() AS server_time"), connection)

In [3]:
df

,server_time
0,2025-10-08 18:03:35.855374+00:00


In [9]:
# how much time does it take to query the last hour of data?
import time
from sqlalchemy import text

COLUMNS = "ts, mmsi, lat, lon, sog, cog, hdg, mt"  # skip 'raw' to avoid JSONB overhead
bbox = None

base_sql = f"""
SELECT {COLUMNS}
FROM ship_positions
WHERE ts >= NOW() - INTERVAL '10 minute'
"""

if bbox is None:
    sql = base_sql + " ORDER BY ts DESC"
    params = {}
else:
    # switch to the ST_Contains/ST_MakeEnvelope
    sql = base_sql + """
      AND lat BETWEEN :min_lat AND :max_lat
      AND lon BETWEEN :min_lon AND :max_lon
      ORDER BY ts DESC
    """
    params = {
        "min_lon": bbox[0], "min_lat": bbox[1],
        "max_lon": bbox[2], "max_lat": bbox[3],
    }

chunksize = 200_000

start = time.perf_counter()
row_count = 0
with engine.connect() as conn:
    # conn.execute(text("SET enable_parallel = on"))
    t = text(sql)
    for chunk in pd.read_sql(t, conn, params=params, chunksize=chunksize):
        row_count += len(chunk)
end = time.perf_counter()

elapsed_s = end - start
print(f"Fetched last hour in {elapsed_s:.3f}s, rows: {row_count:,}")

Fetched last hour in 0.534s, rows: 10,990
